In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.cluster import DBSCAN
from PIL import Image
import numpy as np
import cv2

!pip install lasio
import lasio

In [3]:
df=lasio.read("../input/ovl-las-with-labels/3-BRSA-1194-SES-178_COMPOSITE.las")
well_name=df.well.WELL.value
df=df.df()
df=df.reset_index()
df=df[["DEPTH","GR","DTC","NPHI"]]
df

In [4]:
df.info()

In [5]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns, 'percent_missing': percent_missing})
missing_value_df

In [6]:
df = df.dropna()
df = df.reset_index(drop=True)
df

In [7]:
df.describe()

In [8]:
depth_keypoints=[4568.8808, 4599.818, 4631.3648, 4649.9576, 4701.164, 4736.8256, 4757.0948, 4882.3676, 4894.5596, 4935.7076, 4993.01, 5027.9096, 5050.16, 5077.1348, 5093.2892, 5105.6336, 5135.9612, 5175.128, 5189.1488, 5222.8292, 5237.4596, 5255.7476, 5308.1732, 5328.1376, 5344.9016, 5367.4568]

In [9]:
fig, axes = plt.subplots(figsize=(10,10))

curve_names = ['GR', 'DTC', 'NPHI']

#Set up the plot axes
ax1 = plt.subplot2grid((1,3), (0,0), rowspan=1, colspan = 1) 
ax2 = plt.subplot2grid((1,3), (0,1), rowspan=1, colspan = 1)
ax3 = plt.subplot2grid((1,3), (0,2), rowspan=1, colspan = 1)

#Set up the individual log tracks / subplots
ax1.plot("GR", "DEPTH", data = df, color = "red")
ax2.plot("DTC", "DEPTH", data = df, color = "green")
ax3.plot("NPHI", "DEPTH", data = df, color = "blue")

#Set up the common elements between the subplots
for i, ax in enumerate(fig.axes):
    ax.set_ylim(5500, 4500) # Set the depth range
    ax.grid()
    ax.set_xlabel(curve_names[i])
    for value in depth_keypoints:
        ax.axhline(y=value, color='black', linestyle='-', linewidth=1)

#Hide tick labels on the y-axis 
for ax in [ax2, ax3]:
    plt.setp(ax.get_yticklabels(), visible = False)

#Reduce the space between each subplot
plt.subplots_adjust(wspace = 0)

# Plot a horizontal line using axhline() in pyplot
# for value in depth_keypoints:
#     plt.axhline(y=value, color='r', linestyle='-', linewidth=3)
plt.show()

In [10]:
filtered_df = df.filter(['GR', 'DTC', 'NPHI'])

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,255))
scaled_array = scaler.fit_transform(filtered_df)
scaled_array

In [12]:
scaled_df = pd.DataFrame(scaled_array, columns = ['GR','DTC','NPHI']).astype('int')
scaled_df

In [13]:
corresponding_GR = []
corresponding_DTC = []
corresponding_NPHI = []

y = list(df['DEPTH'])
x1 = list(df['GR'])
x2 = list(df['DTC'])
x3 = list(df['NPHI'])

for value in depth_keypoints:
    val1 = np.interp(value, y, x1)
    val2 = np.interp(value, y, x2)
    val3 = np.interp(value, y, x3)
    corresponding_GR.append(val1)
    corresponding_DTC.append(val2)
    corresponding_NPHI.append(val3)

In [14]:
keypoints_df = pd.DataFrame()
keypoints_df['GR'] = corresponding_GR
keypoints_df['DTC'] = corresponding_DTC
keypoints_df['NPHI'] = corresponding_NPHI
keypoints_df = keypoints_df.astype('int')
keypoints_df

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,255))
scaled_keypoints_array = scaler.fit_transform(keypoints_df)
scaled_keypoints_array

In [16]:
scaled_keypoints_df = pd.DataFrame(scaled_keypoints_array, columns=['GR', 'DTC', 'NPHI']).astype('int')
scaled_keypoints_df

In [17]:
scaled_df_to_array = scaled_df.to_numpy().astype(np.uint8)
img_arr = scaled_df_to_array.reshape(55, 103, 3)
img_arr

In [18]:
original = Image.fromarray(img_arr)
original.save('original.png')

original = cv2.imread("./original.png")
plt.imshow(original)